<h1>Blurring 기법을 이용한 자동차 번호판 비식별화</h1>

In [ ]:
import os
import cv2
import cvlib as cv
from cvlib.object_detection import draw_bbox

In [ ]:
import numpy as np
import imutils

In [ ]:
capture = cv2.VideoCapture("/Users/heoyoonhwe/Documents/dss0/sample.avi")
fps = capture.get(cv2.CAP_PROP_FPS)
w = round(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
h = round(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'DIVX')

In [ ]:
out = cv2.VideoWriter("/Users/heoyoonhwe/Documents/dss0/sample_output.avi", fourcc, fps, (w, h))

In [ ]:
idx = 0
l = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))

In [ ]:
while capture.isOpened():
    if idx % 100 == 0:
        print(idx,"/", l)
        
    ret, frame = capture.read()
    
    if (ret):  #프레임을 제대로 읽었다면
        idx += 1 #idx에 1 더함
        bbox, label, conf = cv.detect_common_objects(frame)
        
        for i, cla in enumerate(label):
            if cla == 'car' or cla == 'bus' or cla == 'truck': #탐지된 라벨이 car, bus, truck인 경우
                deidentification_h = int((bbox[i][3] - bbox[i][1]) * 1/3) #비식별화 처리(blurring)할 영역 설정
                if bbox[i][0] > 0 and bbox[i][1] > 0:
                    deidentification_area = frame[bbox[i][3] - deidentification_h:bbox[i][3] + int(deidentification_h/3), bbox[i][0]:bbox[i][2]]
                    deidentification_img = cv2.blur(deidentification_area, (7,7)) #blurring 수행
                    tmp = frame[bbox[i][3] - deidentification_h:bbox[i][3] + int(deidentification_h/3), bbox[i][0]:bbox[i][2]]
                    frame[bbox[i][3] - deidentification_h:bbox[i][3] + int(deidentification_h/3), bbox[i][0]:bbox[i][2]] = deidentification_img #output 영상에 해당 img 담기
        frame = draw_bbox(frame, bbox, label, conf, write_conf = True)

        ##재식별 코드
        #deidentification_img = tmp #원본을 임시 저장했던 것을 다시 적용
        #frame[bbox[i][3] - deidentification_h:bbox[i][3] + int(deidentification_h/3), bbox[i][0]:bbox[i][2]] = deidentification_img #output 영상에 해당 img 담기
        #frame = draw_bbox(frame, bbox, label, conf, write_conf = True)
        
        # display output
        cv2.namedWindow('frame', cv2.WINDOW_NORMAL) #새로운 윈도우 창(frame)을 띄워 내용 넣기 
        cv2.resizeWindow('frame', w, h) #프레임 창을 사이즈에 맞게 설정
        cv2.imshow("frame", frame) #프레임을 화면에 디스플레이
        out.write(frame)  #output 영상에 프레임을 저장
        if cv2.waitKey(0) & 0xFF == ord('q'): #영상이 종료되거나 q가 입력되면
            break #종료
    else: #프레임을 제대로 읽지 못했다면
        print("프레임 읽지 못함...")
        break #종료

In [ ]:
out.release()
capture.release()
cv2.destroyAllWindows()